**ReadMe**

Please first change the input and output directory before running the code.
To inference the model after saving it, use the codes in the *Inference* section after running the installs and the imports. Make sure to change the model and toikenizer directory to your model directory.

In the *def tokenization_Inf(batched_text):* function, change the *batched_text['text']* to the column name of your X variable.

Depends on whether you have the labels available for the testing set, use either *#dataset_Inf.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])* or *dataset_Inf.set_format('torch', columns=['input_ids', 'attention_mask'])*. If the testing set has been labelled, make sure to change the labelled column name to *label*. Use *#trainer.evaluate(eval_dataset=dataset_Inf)* to evaluate test data with labels.

# Pip Install & Import

In [15]:
'''!pip install datasets
!pip install wandb
!pip install transformers==4.37.2
!pip install accelerate==0.22.0'''

'!pip install datasets\n!pip install wandb\n!pip install transformers==4.37.2\n!pip install accelerate==0.22.0'

In [16]:
import pandas as pd
import datasets
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification,Trainer, TrainingArguments
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split

import pandas as pd
from tqdm import tqdm
import wandb
import os
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Preprocessing

In [17]:
ori_data = pd.read_csv('/content/drive/MyDrive/95885 DSBD/95885_Project2_T15/dataset.csv', encoding='latin1')
column_names = ['target', 'ID', 'Date', 'Query', 'User', 'text']
ori_data.columns = column_names
ori_data.head()

,target,ID,Date,Query,User,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [18]:
data = pd.concat([
    ori_data[ori_data['target'] == 0].sample(n=10000, random_state=42),
    ori_data[ori_data['target'] == 4].sample(n=10000, random_state=42)
])
data['label'] = data['target'].apply(lambda x: 1 if x == 0 else 0)
data = data[['text', 'label']]
dataset = datasets.Dataset.from_pandas(data)
df_30, df_70 = train_test_split(data, test_size=0.7, random_state=777)
df_30_dataset = datasets.Dataset.from_pandas(df_30)
df_70_dataset = datasets.Dataset.from_pandas(df_70)

In [19]:
data.head()

,text,label
212188,@dickiemaxx Win a few; lose few a with a,1
299036,@lavjonas tokio hotel and won tokio hotel that...,1
475977,"Right, off to work Only 5 hours to go until I...",1
385829,@daibarnes and guess who's landed that plum jo...,1
138859,@rabbit_rabid can't sleep... come give me hug...,1


In [20]:
# load model and tokenizer and define length of the text sequence
model = RobertaForSequenceClassification.from_pretrained('roberta-base')
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', max_length = 512)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
# define a function that will tokenize the model, and will return the relevant inputs for the model
def tokenization(batched_text):
    return tokenizer(batched_text['text'], padding = True, truncation=True)


train_data = df_70_dataset.map(tokenization, batched = True, batch_size = len(df_70))
test_data = df_30_dataset.map(tokenization, batched = True, batch_size = len(df_30))

Map:   0%|          | 0/14000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [22]:
train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# Building & Training Model

In [23]:
# define accuracy metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [24]:
import accelerate

# define the training arguments
training_args = TrainingArguments(
    output_dir = '/content/drive/MyDrive/HYP/BadLanguage',
    num_train_epochs=5,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 16,
    per_device_eval_batch_size= 8,
    evaluation_strategy = "epoch",
    save_strategy='epoch',
    disable_tqdm = False,
    load_best_model_at_end=True,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps = 8,
    fp16 = True,
    logging_dir='/content/drive/MyDrive/HYP/BadLanguage',
    dataloader_num_workers = 8,
    run_name = 'roberta-classification'
)

In [25]:
# instantiate the trainer class and check for available devices
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:433: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


'cuda'

In [26]:
# train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,0.420900,0.437862,0.813667,0.829833,0.760815,0.912621
1,0.383400,0.354621,0.851000,0.851297,0.845950,0.856712
2,0.288900,0.386953,0.845833,0.841907,0.859986,0.824573
4,0.093600,0.550743,0.851667,0.852307,0.845015,0.859725


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Checkpoint destination directory /content/drive/MyDrive/HYP/BadLanguage/checkpoint-437 already exists and is non-empty.Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower t

TrainOutput(global_step=1090, training_loss=0.30078294408430745, metrics={'train_runtime': 2340.6091, 'train_samples_per_second': 29.907, 'train_steps_per_second': 0.466, 'total_flos': 1.35867260099712e+16, 'train_loss': 0.30078294408430745, 'epoch': 4.98})

In [27]:
trainer.evaluate(eval_dataset=test_data)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


{'eval_loss': 0.354621022939682,
 'eval_accuracy': 0.851,
 'eval_f1': 0.8512974051896207,
 'eval_precision': 0.8459504132231405,
 'eval_recall': 0.8567124204887847,
 'eval_runtime': 32.4334,
 'eval_samples_per_second': 184.994,
 'eval_steps_per_second': 23.124,
 'epoch': 4.98}

In [28]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


{'eval_loss': 0.354621022939682,
 'eval_accuracy': 0.851,
 'eval_f1': 0.8512974051896207,
 'eval_precision': 0.8459504132231405,
 'eval_recall': 0.8567124204887847,
 'eval_runtime': 46.8599,
 'eval_samples_per_second': 128.041,
 'eval_steps_per_second': 16.005,
 'epoch': 4.98}